In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('../../data/焚化廠資料.xlsx')
df_g = df.groupby('年份')
df

,焚化廠名稱,縣市,年份,其他成本,勞動成本,焚化處理量,設計處理量,設計發電量,實際發電量,二氧化碳當量
0,宜蘭縣利澤垃圾資源回收(焚化)廠,宜蘭縣,2020,960330.0,1033323.0,182263.4,600,14700000,99631.87,136697.550
1,宜蘭縣利澤垃圾資源回收(焚化)廠,宜蘭縣,2011,329010.0,468548.0,199732.2,600,14700000,107838.70,149799.150
2,宜蘭縣利澤垃圾資源回收(焚化)廠,宜蘭縣,2019,846509.0,999635.0,202111.2,600,14700000,107890.40,151583.400
3,宜蘭縣利澤垃圾資源回收(焚化)廠,宜蘭縣,2012,371861.0,518953.0,202728.5,600,14700000,110156.70,152046.375
4,宜蘭縣利澤垃圾資源回收(焚化)廠,宜蘭縣,2013,411821.0,587162.0,205014.9,600,14700000,110901.60,153761.175
...,...,...,...,...,...,...,...,...,...,...
235,臺南市城西垃圾焚化廠,臺南市,2014,1085896.0,1285597.5,219334.7,900,14300000,91008.24,164501.025
236,臺南市城西垃圾焚化廠,臺南市,2016,1196041.0,1387306.0,219850.9,900,14300000,92339.52,164888.175
237,臺南市城西垃圾焚化廠,臺南市,2015,1096999.5,1299537.5,221728.8,900,14300000,93911.04,166296.600
238,臺南市城西垃圾焚化廠,臺南市,2011,835958.0,999123.0,222657.9,900,14300000,91025.38,166993.425


In [3]:
population = pd.read_csv('縣市.csv')
population.index = list(population[population.columns[0]])
population = population.drop(population.columns[0], axis = 1)
population

,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
新北市,NaN,NaN,3954929.0,3966818,3970644,3979208,3986689,3995717,4018696,4030954
臺北市,NaN,NaN,2686516.0,2702315,2704810,2695704,2683257,2668572,2645014,2602418
桃園市,NaN,NaN,2701661.0,2058328,2105780,2147763,2188017,2220872,2249037,2268807
臺中市,NaN,NaN,NaN,2719835,2744445,2767239,2787070,2803894,2815261,2820787
臺南市,NaN,NaN,NaN,1884284,1885541,1886033,1886522,1883831,1880906,1874917
高雄市,NaN,NaN,NaN,2778992,2778918,2779371,2776912,2773533,2773198,2765932
宜蘭縣,NaN,NaN,NaN,458777,458117,457538,456607,455221,454178,453087
新竹市,NaN,NaN,NaN,537630,542042,547481,552169,557010,563933,451412
苗栗縣,NaN,NaN,NaN,567132,563912,559189,553807,548863,545459,542590
彰化縣,NaN,NaN,NaN,1291474,1289072,1287146,1282458,1277824,1272802,1266670


In [4]:
def egalitarianism(C, year = False):
    r = {}
    if year:
        years = range(year, year+1)
    else:
        years = range(2014, 2021)
    for y in years:
        Y = df_g.get_group(y)
        vc = Y['縣市'].value_counts()
        temp = []
        for n in range(Y.shape[0]):
            temp.append(population[str(y)][Y.iloc[n]['縣市']]/vc[Y.iloc[n]['縣市']])
        temp = np.array(temp)/sum(temp)
        Y['分配碳權'] = C * temp
        Y.index = range(Y.shape[0])
        r[y] = Y[['焚化廠名稱', '分配碳權']]
    if year:
        return r[year]
    else:
        return r

In [5]:
with pd.ExcelWriter('egalitarianism.xlsx') as writer:
    for y in range(2014, 2021):
        egalitarianism(sum(df_g.get_group(y)['二氧化碳當量']), y).to_excel(writer, sheet_name = str(y), encoding = 'utf_8_sig')

C:\Users\lolos\AppData\Local\Temp/ipykernel_5052/619268500.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y['分配碳權'] = C * temp
